# Evaluating checkpoints with DuckDB

References:

- https://medium.com/@anshubantra/using-duckdb-in-python-a-comprehensive-guide-d14bc0b06546

In [1]:
import duckdb
import json
from pathlib import Path
import glob

## Functions

In [84]:
def upsert_table_from_json(conn, suffix_key_pattern, prefix_key_pattern, table_name, source, key_value, drop, content_key, alias, doc_id_key="id") -> bool:

    _data = []
    _source_file = Path(source).name
    directory_path = Path("temp")

    # Combine the directory path and the filename
    _temp_file = directory_path / _source_file

    with open(source, "r") as f:
        loaded_data = json.load(f)

    keys = loaded_data.keys()

    for key in keys:
        if key.startswith(prefix_key_pattern) and key.endswith(suffix_key_pattern):
            print(key)
            _data.append(loaded_data[key])

    if not _data:
        print("No data found. Check the key pattern.")
        return False
    
    with open(_temp_file, "w") as f:
        json.dump(_data, f, indent=4)

    if drop:
        _query_drop_table = f"""
        DROP TABLE {table_name};
        """

        print(f"{_query_drop_table=}")
        print(f"Trying to drop table because drop parameter is {drop}")

        try:
            conn.execute(_query_drop_table)
            print(f"Table {table_name} dropped")
        except duckdb.CatalogException as e:
            print(e)

    _query_insert_data = f"""
    INSERT INTO {table_name} ({doc_id_key}, prompt, file_source, {alias}, created_at)
    SELECT 
        {doc_id_key}, 
        '{key_value}' as prompt, 
        '{_source_file}' as file_source, 
        unnest({content_key}) as {alias},
        now() as created_at
    FROM 
        read_json_auto("{_temp_file}");
    """

    print(f"{_query_insert_data=}")
    print(f"Trying to insert into {table_name}")

    try:
        conn.execute(_query_insert_data)
        print(f"Data inserted into {table_name}.")
    except duckdb.CatalogException as e:
        print(e)
        print(f"Failed to insert, trying create {table_name}")
        _query_create_table = f"""
        CREATE TABLE {table_name} AS
        SELECT {doc_id_key}, 
        '{key_value}' as prompt, 
        '{_source_file}' as file_source, 
        unnest({content_key}) as {alias},
        now() as 'created_at'
        FROM read_json_auto("{_temp_file}");
        """
        try:
            print(f"{_query_create_table=}")
            conn.execute(_query_create_table)
            print(f"Table {table_name} created and loaded.")
        except duckdb.CatalogException as e:
            print(e)
            print(f"Failed to create {table_name}")
            return False

    return True

## Connect database

In [3]:
conn = duckdb.connect("cfr2sbvr_db/database.db")

## True tables

In [37]:
upsert_table_from_json(
    conn,
    suffix_key_pattern="_P1|true_table",
    prefix_key_pattern="§",
    table_name="EXTRACTED_ELEMENTS_TRUE",
    source="cfr2sbvr_db/true/documents_true_table.json",
    key_value="extract_p1",
    drop=True,
    content_key="content.elements",
    alias="elements",
    doc_id_key="id"

)

§ 275.0-2_P1|true_table
§ 275.0-5_P1|true_table
§ 275.0-7_P1|true_table
_query_drop_table='\n        DROP TABLE EXTRACTED_ELEMENTS_TRUE;\n        '
Trying to drop table because drop parameter is True
Table EXTRACTED_ELEMENTS_TRUE dropped
_query_insert_data='\n    INSERT INTO EXTRACTED_ELEMENTS_TRUE (id, prompt, file_source, content.elements, created_at)\n    SELECT \n        id, \n        \'extract_p1\' as prompt, \n        \'documents_true_table.json\' as file_source, \n        unnest(content.elements) as elements,\n        now() as created_at\n    FROM \n        read_json_auto("temp/documents_true_table.json");\n    '
Trying to insert into EXTRACTED_ELEMENTS_TRUE
Catalog Error: Table with name EXTRACTED_ELEMENTS_TRUE does not exist!
Did you mean "database.EXTRACTED_ELEMENTS_TRUE"?
Failed to insert, trying create EXTRACTED_ELEMENTS_TRUE
_query_create_table='\n        CREATE TABLE EXTRACTED_ELEMENTS_TRUE AS\n        SELECT id, \n        \'extract_p1\' as prompt, \n        \'documents_t

True

In [38]:
query = """
SELECT * FROM EXTRACTED_ELEMENTS_TRUE;
"""

conn.sql(query)

┌──────────────┬────────────┬───────────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Total verb symbols extracted

In [40]:
query = """
SELECT
    COUNT(verb) AS total_verbs
FROM (
    SELECT
        UNNEST(CAST(json_extract(elements, '$.verb_symbols') AS VARCHAR[])) AS verb
    FROM
        EXTRACTED_ELEMENTS_TRUE
) AS flattened_verbs;
"""

conn.sql(query)

┌─────────────┐
│ total_verbs │
│    int64    │
├─────────────┤
│          61 │
└─────────────┘

distinct verb symbols

In [41]:
query = """
SELECT
    COUNT(DISTINCT verb) AS distinct_verbs_count
FROM (
    SELECT
        UNNEST(CAST(json_extract(elements, '$.verb_symbols') AS VARCHAR[])) AS verb
    FROM
        EXTRACTED_ELEMENTS_TRUE
) AS flattened_verbs;
"""

conn.sql(query)

┌──────────────────────┐
│ distinct_verbs_count │
│        int64         │
├──────────────────────┤
│                   43 │
└──────────────────────┘

In [42]:
query = """
SELECT DISTINCT
    verb,
    doc_id,
    source
FROM (
    SELECT
        UNNEST(CAST(json_extract(elements, '$.verb_symbols') AS VARCHAR[])) AS verb,
        id AS doc_id,
        CAST(json_extract(elements, '$.sources') AS VARCHAR) AS source
    FROM
        EXTRACTED_ELEMENTS_TRUE
) AS distinct_combinations;
"""

true_values=conn.sql(query).fetchall()

conn.sql(query)



┌────────────────────────┬──────────────┬─────────────────┐
│          verb          │    doc_id    │     source      │
│        varchar         │   varchar    │     varchar     │
├────────────────────────┼──────────────┼─────────────────┤
│ by serving             │ § 275.0-2_P1 │ ["(a)"]         │
│ serve                  │ § 275.0-2_P1 │ ["(a)(1)"]      │
│ by                     │ § 275.0-2_P1 │ ["(a)(2)"]      │
│ certifies              │ § 275.0-2_P1 │ ["(a)(3)"]      │
│ constitutes            │ § 275.0-2_P1 │ ["(a)(3)"]      │
│ participates           │ § 275.0-2_P1 │ ["(b)(1)"]      │
│ is incorporated        │ § 275.0-2_P1 │ ["(b)(2)"]      │
│ has                    │ § 275.0-2_P1 │ ["(b)(3)"]      │
│ will indicate          │ § 275.0-5_P1 │ ["(a)"]         │
│ will be issued         │ § 275.0-5_P1 │ ["(b)"]         │
│       ·                │      ·       │    ·            │
│       ·                │      ·       │    ·            │
│       ·                │      ·       

## Predict tables (P1)

In [51]:
# Define the directory and pattern
directory = Path("cfr2sbvr_db/extraction")
pattern = "documents-*.json"

# Use glob to find all files matching the pattern
files = directory.glob(pattern)

drop=True # Drop the table first time
for file_path in files:
    print(file_path)  # Output each file path
    upsert_table_from_json(
        conn,
        suffix_key_pattern="_P1|llm_response",
        prefix_key_pattern="§",
        table_name="EXTRACTED_ELEMENTS",
        source=file_path,
        key_value="extract_p1",
        drop=drop,
        content_key="content.elements",
        alias="elements",
        doc_id_key="id"
    )
    drop=False # Stop dropping tables

cfr2sbvr_db/extraction/documents-2024-11-29-4.json
§ 275.0-2_P1|llm_response
§ 275.0-5_P1|llm_response
§ 275.0-7_P1|llm_response
_query_drop_table='\n        DROP TABLE EXTRACTED_ELEMENTS;\n        '
Trying to drop table because drop parameter is True
Table EXTRACTED_ELEMENTS dropped
_query_insert_data='\n    INSERT INTO EXTRACTED_ELEMENTS (id, prompt, file_source, elements, created_at)\n    SELECT \n        id, \n        \'extract_p1\' as prompt, \n        \'documents-2024-11-29-4.json\' as file_source, \n        unnest(content.elements) as elements,\n        now() as created_at\n    FROM \n        read_json_auto("temp/documents-2024-11-29-4.json");\n    '
Trying to insert into EXTRACTED_ELEMENTS
Catalog Error: Table with name EXTRACTED_ELEMENTS does not exist!
Did you mean "database.EXTRACTED_ELEMENTS"?
Failed to insert, trying create EXTRACTED_ELEMENTS
_query_create_table='\n        CREATE TABLE EXTRACTED_ELEMENTS AS\n        SELECT id, \n        \'extract_p1\' as prompt, \n        

In [52]:
query = """
SELECT * FROM EXTRACTED_ELEMENTS;
"""

conn.sql(query)

┌──────────────┬────────────┬─────────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Total verb symbols extracted

In [53]:
query = """
SELECT
    COUNT(verb) AS total_verbs
FROM (
    SELECT
        UNNEST(CAST(json_extract(elements, '$.verb_symbols') AS VARCHAR[])) AS verb
    FROM
        EXTRACTED_ELEMENTS
) AS flattened_verbs;
"""

conn.sql(query)

┌─────────────┐
│ total_verbs │
│    int64    │
├─────────────┤
│         550 │
└─────────────┘

distinct verb symbols

In [54]:
query = """
SELECT
    COUNT(DISTINCT verb) AS distinct_verbs_count
FROM (
    SELECT
        UNNEST(CAST(json_extract(elements, '$.verb_symbols') AS VARCHAR[])) AS verb
    FROM
        EXTRACTED_ELEMENTS
) AS flattened_verbs;
"""

conn.sql(query)

┌──────────────────────┐
│ distinct_verbs_count │
│        int64         │
├──────────────────────┤
│                   81 │
└──────────────────────┘

Distinct verb symbols with doc_id and source

In [55]:
query = """
SELECT DISTINCT
    verb,
    doc_id,
    source
FROM (
    SELECT
        UNNEST(CAST(json_extract(elements, '$.verb_symbols') AS VARCHAR[])) AS verb,
        id AS doc_id,
        CAST(json_extract(elements, '$.sources') AS VARCHAR) AS source
    FROM
        EXTRACTED_ELEMENTS
) AS distinct_combinations;
"""

#pred_values=duckdb.sql(query).fetchall()

conn.sql(query)



┌────────────────────┬──────────────┬────────────────────┐
│        verb        │    doc_id    │       source       │
│      varchar       │   varchar    │      varchar       │
├────────────────────┼──────────────┼────────────────────┤
│ by serving         │ § 275.0-2_P1 │ ["(a)"]            │
│ serve              │ § 275.0-2_P1 │ ["(a)(1)"]         │
│ by                 │ § 275.0-2_P1 │ ["(a)(2)"]         │
│ certifies          │ § 275.0-2_P1 │ ["(a)(3)"]         │
│ constitutes        │ § 275.0-2_P1 │ ["(a)(3)"]         │
│ participates       │ § 275.0-2_P1 │ ["(b)(1)"]         │
│ is incorporated    │ § 275.0-2_P1 │ ["(b)(2)"]         │
│ has                │ § 275.0-2_P1 │ ["(b)(3)"]         │
│ will indicate      │ § 275.0-5_P1 │ ["(a)"]            │
│ will be issued     │ § 275.0-5_P1 │ ["(b)"]            │
│   ·                │      ·       │    ·               │
│   ·                │      ·       │    ·               │
│   ·                │      ·       │    ·              

Verb symbols with doc_id and source for evaluation

In [56]:
query = """
SELECT
    verb,
    doc_id,
    source
FROM (
    SELECT
        UNNEST(CAST(json_extract(elements, '$.verb_symbols') AS VARCHAR[])) AS verb,
        id AS doc_id,
        CAST(json_extract(elements, '$.sources') AS VARCHAR) AS source
    FROM
        EXTRACTED_ELEMENTS
) AS distinct_combinations;
"""

pred_values=conn.sql(query).fetchall()

conn.sql(query)



┌────────────────────────┬──────────────┬─────────────────┐
│          verb          │    doc_id    │     source      │
│        varchar         │   varchar    │     varchar     │
├────────────────────────┼──────────────┼─────────────────┤
│ serve                  │ § 275.0-2_P1 │ ["(a)"]         │
│ by serving             │ § 275.0-2_P1 │ ["(a)"]         │
│ serve                  │ § 275.0-2_P1 │ ["(a)(1)"]      │
│ by furnishing          │ § 275.0-2_P1 │ ["(a)(1)"]      │
│ with                   │ § 275.0-2_P1 │ ["(a)(1)"]      │
│ will forward           │ § 275.0-2_P1 │ ["(a)(2)"]      │
│ by                     │ § 275.0-2_P1 │ ["(a)(2)"]      │
│ at                     │ § 275.0-2_P1 │ ["(a)(2)"]      │
│ certifies              │ § 275.0-2_P1 │ ["(a)(3)"]      │
│ was served             │ § 275.0-2_P1 │ ["(a)(3)"]      │
│     ·                  │      ·       │     ·           │
│     ·                  │      ·       │     ·           │
│     ·                  │      ·       

## Evaluation

Evaluate with distinct pred and true verb symbols extraction

In [17]:
# Convert lists to sets
true_set = set(true_values)
predicted_set = set(pred_values)

# Calculate True Positives (TP)
true_positives = true_set.intersection(predicted_set)
TP = len(true_positives)

# Calculate False Positives (FP)
false_positives = predicted_set.difference(true_set)
FP = len(false_positives)

# Calculate False Negatives (FN)
false_negatives = true_set.difference(predicted_set)
FN = len(false_negatives)

# Calculate Precision, Recall, and F1-Score
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1_score:.2f}")

# Find difference: elements in predicted_set but not in true_set
predicted_not_in_true = predicted_set.difference(true_set)

# Count elements not in true values
count_predicted_not_in_true = len(predicted_not_in_true)

print(f"Number of predicted values not in true values: {count_predicted_not_in_true}")

# Find intersection
common_elements = true_set.intersection(predicted_set)

# Count common elements
count_common = len(common_elements)

print(f"Number of true values in predicted values: {count_common}")

Precision: 0.49
Recall: 1.00
F1-Score: 0.66
Number of predicted values not in true values: 63
Number of true values in predicted values: 61


Evaluation with all pred values.

In [18]:
# Calculate True Positives (TP)
TP = sum(1 for item in pred_values if item in true_values)

# Calculate False Positives (FP)
FP = sum(1 for item in pred_values if item not in true_values)

# Calculate False Negatives (FN)
FN = sum(1 for item in true_values if item not in pred_values)

# Calculate Precision, Recall, and F1-Score
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1_score:.2f}")

print(f"{TP=}, {FP=}, {FN=}")

Precision: 0.80
Recall: 1.00
F1-Score: 0.89
TP=442, FP=108, FN=0


## Evaluation Terms and Names definitions (P2)

### True table

In [57]:
upsert_table_from_json(
    conn,
    suffix_key_pattern="_P2|true_table",
    prefix_key_pattern="§",
    table_name="EXTRACTED_TERMS_TRUE",
    source="cfr2sbvr_db/true/documents_true_table.json",
    key_value="extract_p2",
    drop=True,
    content_key="content.terms",
    alias="terms",
    doc_id_key="id"
)

§ 275.0-2_P2|true_table
§ 275.0-5_P2|true_table
§ 275.0-7_P2|true_table
_query_drop_table='\n        DROP TABLE EXTRACTED_TERMS_TRUE;\n        '
Trying to drop table because drop parameter is True
Table EXTRACTED_TERMS_TRUE dropped
_query_insert_data='\n    INSERT INTO EXTRACTED_TERMS_TRUE (id, prompt, file_source, terms, created_at)\n    SELECT \n        id, \n        \'extract_p2\' as prompt, \n        \'documents_true_table.json\' as file_source, \n        unnest(content.terms) as terms,\n        now() as created_at\n    FROM \n        read_json_auto("temp/documents_true_table.json");\n    '
Trying to insert into EXTRACTED_TERMS_TRUE
Catalog Error: Table with name EXTRACTED_TERMS_TRUE does not exist!
Did you mean "database.EXTRACTED_TERMS_TRUE"?
Failed to insert, trying create EXTRACTED_TERMS_TRUE
_query_create_table='\n        CREATE TABLE EXTRACTED_TERMS_TRUE AS\n        SELECT id, \n        \'extract_p2\' as prompt, \n        \'documents_true_table.json\' as file_source, \n      

True

In [58]:
query = """
SELECT * FROM EXTRACTED_TERMS_TRUE;
"""

conn.sql(query)

┌──────────────┬────────────┬───────────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────────────────┐
│      id      │   prompt   │        file_source        │                                                                                                                                            terms                                                                                                                                             │         created_at         │
│   varchar    │  varchar   │          varchar          │                                                                                                                struct(term varchar, definition varchar, islocalscope boolean)                     

Total terms per id

In [59]:
query = """
SELECT
    id, COUNT(terms) AS total_terms, COUNT(DISTINCT terms) AS total_terms_distinct
FROM EXTRACTED_TERMS_TRUE
GROUP BY id;
"""

conn.sql(query)

┌──────────────┬─────────────┬──────────────────────┐
│      id      │ total_terms │ total_terms_distinct │
│   varchar    │    int64    │        int64         │
├──────────────┼─────────────┼──────────────────────┤
│ § 275.0-7_P2 │          31 │                   31 │
│ § 275.0-5_P2 │          22 │                   22 │
│ § 275.0-2_P2 │          31 │                   31 │
└──────────────┴─────────────┴──────────────────────┘

terms

In [60]:
query = """
SELECT
    id, terms.term as term, terms.definition as definition, if(terms.definition is NULL, 0, 1) as hasDefinition, terms.isLocalScope as isLocalScope
FROM EXTRACTED_TERMS_TRUE
-- GROUP BY id;
"""

conn.sql(query)

┌──────────────┬────────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────┬──────────────┐
│      id      │                  term                  │                                                                                                            definition                                                                                                            │ hasDefinition │ isLocalScope │
│   varchar    │                varchar                 │                                                                                                             varchar                                                                                                              │     int32     │   boolean    │
├──────────────┼────────────────────────────────────

How many terms has definition

In [61]:
query = """
SELECT
    id, count(terms) as term, count(if(terms.definition is NULL, NULL, 1)) as hasDefinition, count(if(terms.definition is NULL, NULL, 1)) / count(terms) as ratio
FROM EXTRACTED_TERMS_TRUE
GROUP BY id
ORDER BY id
;
"""

conn.sql(query)

┌──────────────┬───────┬───────────────┬─────────────────────┐
│      id      │ term  │ hasDefinition │        ratio        │
│   varchar    │ int64 │     int64     │       double        │
├──────────────┼───────┼───────────────┼─────────────────────┤
│ § 275.0-2_P2 │    31 │             9 │  0.2903225806451613 │
│ § 275.0-5_P2 │    22 │            19 │  0.8636363636363636 │
│ § 275.0-7_P2 │    31 │             8 │ 0.25806451612903225 │
└──────────────┴───────┴───────────────┴─────────────────────┘

In [62]:
query = """
SELECT
    id, count(terms) as term, terms.isLocalScope as isLocalScope
FROM EXTRACTED_TERMS_TRUE
GROUP BY id, terms.isLocalScope
ORDER BY id
;
"""

conn.sql(query)

┌──────────────┬───────┬──────────────┐
│      id      │ term  │ isLocalScope │
│   varchar    │ int64 │   boolean    │
├──────────────┼───────┼──────────────┤
│ § 275.0-2_P2 │     7 │ true         │
│ § 275.0-2_P2 │    24 │ false        │
│ § 275.0-5_P2 │    18 │ true         │
│ § 275.0-5_P2 │     1 │ NULL         │
│ § 275.0-5_P2 │     3 │ false        │
│ § 275.0-7_P2 │     8 │ true         │
│ § 275.0-7_P2 │    23 │ false        │
└──────────────┴───────┴──────────────┘

isLocalScope per document

In [63]:
query = """
WITH TotalCounts AS (
    SELECT
        id,
        COUNT(terms) AS total_terms
    FROM EXTRACTED_TERMS_TRUE
    GROUP BY id
)
SELECT
    e.id,
    COUNT(e.terms) AS term,
    e.terms.isLocalScope AS isLocalScope,
    (COUNT(e.terms) * 100.0 / tc.total_terms) AS percentage
FROM EXTRACTED_TERMS_TRUE e
JOIN TotalCounts tc ON e.id = tc.id
GROUP BY e.id, e.terms.isLocalScope, tc.total_terms
ORDER BY e.id, e.terms.isLocalScope;
"""

conn.sql(query)

┌──────────────┬───────┬──────────────┬────────────────────┐
│      id      │ term  │ isLocalScope │     percentage     │
│   varchar    │ int64 │   boolean    │       double       │
├──────────────┼───────┼──────────────┼────────────────────┤
│ § 275.0-2_P2 │    24 │ false        │  77.41935483870968 │
│ § 275.0-2_P2 │     7 │ true         │ 22.580645161290324 │
│ § 275.0-5_P2 │     3 │ false        │ 13.636363636363637 │
│ § 275.0-5_P2 │    18 │ true         │  81.81818181818181 │
│ § 275.0-5_P2 │     1 │ NULL         │  4.545454545454546 │
│ § 275.0-7_P2 │    23 │ false        │  74.19354838709677 │
│ § 275.0-7_P2 │     8 │ true         │ 25.806451612903224 │
└──────────────┴───────┴──────────────┴────────────────────┘

Total terms per id

In [64]:
query = """
SELECT
    id, COUNT(terms) AS total_terms, COUNT(DISTINCT terms) AS total_terms_distinct
FROM EXTRACTED_TERMS_TRUE
GROUP BY id;
"""

conn.sql(query)

┌──────────────┬─────────────┬──────────────────────┐
│      id      │ total_terms │ total_terms_distinct │
│   varchar    │    int64    │        int64         │
├──────────────┼─────────────┼──────────────────────┤
│ § 275.0-5_P2 │          22 │                   22 │
│ § 275.0-7_P2 │          31 │                   31 │
│ § 275.0-2_P2 │          31 │                   31 │
└──────────────┴─────────────┴──────────────────────┘

terms

In [65]:
query = """
SELECT
    id, terms.term as term, terms.definition as definition, if(terms.definition is NULL, 0, 1) as hasDefinition, terms.isLocalScope as isLocalScope
FROM EXTRACTED_TERMS_TRUE
-- GROUP BY id;
"""

conn.sql(query)

┌──────────────┬────────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────┬──────────────┐
│      id      │                  term                  │                                                                                                            definition                                                                                                            │ hasDefinition │ isLocalScope │
│   varchar    │                varchar                 │                                                                                                             varchar                                                                                                              │     int32     │   boolean    │
├──────────────┼────────────────────────────────────

How many terms has definition

In [66]:
query = """
SELECT
    id, count(terms) as term, count(if(terms.definition is NULL, NULL, 1)) as hasDefinition, count(if(terms.definition is NULL, NULL, 1)) / count(terms) as ratio
FROM EXTRACTED_TERMS_TRUE
GROUP BY id
ORDER BY id
;
"""

conn.sql(query)

┌──────────────┬───────┬───────────────┬─────────────────────┐
│      id      │ term  │ hasDefinition │        ratio        │
│   varchar    │ int64 │     int64     │       double        │
├──────────────┼───────┼───────────────┼─────────────────────┤
│ § 275.0-2_P2 │    31 │             9 │  0.2903225806451613 │
│ § 275.0-5_P2 │    22 │            19 │  0.8636363636363636 │
│ § 275.0-7_P2 │    31 │             8 │ 0.25806451612903225 │
└──────────────┴───────┴───────────────┴─────────────────────┘

In [29]:
query = """
SELECT
    id, count(terms) as term, terms.isLocalScope as isLocalScope
FROM EXTRACTED_TERMS_TRUE
GROUP BY id, terms.isLocalScope
ORDER BY id
;
"""

conn.sql(query)

┌──────────────┬───────┬──────────────┐
│      id      │ term  │ isLocalScope │
│   varchar    │ int64 │   boolean    │
├──────────────┼───────┼──────────────┤
│ § 275.0-2_P2 │     7 │ true         │
│ § 275.0-2_P2 │    24 │ false        │
│ § 275.0-5_P2 │    18 │ true         │
│ § 275.0-5_P2 │     3 │ false        │
│ § 275.0-5_P2 │     1 │ NULL         │
│ § 275.0-7_P2 │     8 │ true         │
│ § 275.0-7_P2 │    23 │ false        │
└──────────────┴───────┴──────────────┘

In [30]:
query = """
SELECT
    id, terms, terms.isLocalScope as isLocalScope
FROM EXTRACTED_TERMS_TRUE
WHERE terms.isLocalScope is NULL
;
"""

conn.sql(query)

┌──────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────┐
│      id      │                                                        terms                                                        │ isLocalScope │
│   varchar    │                           struct(term varchar, definition varchar, islocalscope boolean)                            │   boolean    │
├──────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼──────────────┤
│ § 275.0-5_P2 │ {'term': Motion, 'definition': A formal proposal by the Commission to initiate a proceeding., 'isLocalScope': NULL} │ NULL         │
└──────────────┴─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┴──────────────┘

In [31]:
query = """
SELECT
    id, terms, terms.isLocalScope as isLocalScope
FROM EXTRACTED_TERMS_TRUE
WHERE terms.isLocalScope is NULL
;
"""

conn.sql(query)

┌──────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────┐
│      id      │                                                        terms                                                        │ isLocalScope │
│   varchar    │                           struct(term varchar, definition varchar, islocalscope boolean)                            │   boolean    │
├──────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼──────────────┤
│ § 275.0-5_P2 │ {'term': Motion, 'definition': A formal proposal by the Commission to initiate a proceeding., 'isLocalScope': NULL} │ NULL         │
└──────────────┴─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┴──────────────┘

### Predict table

In [67]:
# Define the directory and pattern
directory = Path("cfr2sbvr_db/extraction")
pattern = "documents-*.json"

# Use glob to find all files matching the pattern
files = directory.glob(pattern)

drop=True # Drop the table first time
for file_path in files:
    print(file_path)  # Output each file path
    upsert_table_from_json(
        conn,
        suffix_key_pattern="_P2|llm_response",
        prefix_key_pattern="§",
        table_name="EXTRACTED_TERMS",
        source=file_path,
        key_value="extract_p2",
        drop=drop,
        content_key="content.terms",
        alias="terms",
        doc_id_key="id"
    )
    drop=False # Stop dropping tables

cfr2sbvr_db/extraction/documents-2024-11-29-4.json
§ 275.0-2_P2|llm_response
§ 275.0-5_P2|llm_response
§ 275.0-7_P2|llm_response
_query_drop_table='\n        DROP TABLE EXTRACTED_TERMS;\n        '
Trying to drop table because drop parameter is True
Table EXTRACTED_TERMS dropped
_query_insert_data='\n    INSERT INTO EXTRACTED_TERMS (id, prompt, file_source, terms, created_at)\n    SELECT \n        id, \n        \'extract_p2\' as prompt, \n        \'documents-2024-11-29-4.json\' as file_source, \n        unnest(content.terms) as terms,\n        now() as created_at\n    FROM \n        read_json_auto("temp/documents-2024-11-29-4.json");\n    '
Trying to insert into EXTRACTED_TERMS
Catalog Error: Table with name EXTRACTED_TERMS does not exist!
Did you mean "database.EXTRACTED_TERMS"?
Failed to insert, trying create EXTRACTED_TERMS
_query_create_table='\n        CREATE TABLE EXTRACTED_TERMS AS\n        SELECT id, \n        \'extract_p2\' as prompt, \n        \'documents-2024-11-29-4.json\' a

In [68]:
query = """
SELECT * FROM EXTRACTED_TERMS;
"""

conn.sql(query)

┌──────────────┬────────────┬─────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────────────────┐
│      id      │   prompt   │         file_source         │                                                                                                                       

Total verb symbols extracted

In [69]:
query = """
SELECT
    id, COUNT(terms) AS total_terms, COUNT(DISTINCT terms) AS total_terms_distinct
FROM EXTRACTED_TERMS
GROUP BY id;
"""

conn.sql(query)

┌──────────────┬─────────────┬──────────────────────┐
│      id      │ total_terms │ total_terms_distinct │
│   varchar    │    int64    │        int64         │
├──────────────┼─────────────┼──────────────────────┤
│ § 275.0-7_P2 │         245 │                  180 │
│ § 275.0-2_P2 │         224 │                  154 │
│ § 275.0-5_P2 │         167 │                  149 │
└──────────────┴─────────────┴──────────────────────┘

distinct verb symbols

In [70]:
query = """
SELECT
    id, terms.term as term, terms.definition as definition, if(terms.definition is NULL, 0, 1) as hasDefinition, terms.isLocalScope as isLocalScope
FROM EXTRACTED_TERMS
-- GROUP BY id;
"""

conn.sql(query)

┌──────────────┬───────────────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────┬──────────────┐
│      id      │               term                │                                                                                                                                                                                               definition                                                                                                                                                                                                │ hasDefinition │ isLocalScope │
│   varchar    │              varchar       

How many terms has definition

In [71]:
query = """
SELECT
    id, count(terms) as term, count(if(terms.definition is NULL, NULL, 1)) as hasDefinition, count(if(terms.definition is NULL, NULL, 1)) / count(terms) as ratio
FROM EXTRACTED_TERMS
GROUP BY id
ORDER BY id
;
"""

conn.sql(query)

┌──────────────┬───────┬───────────────┬─────────────────────┐
│      id      │ term  │ hasDefinition │        ratio        │
│   varchar    │ int64 │     int64     │       double        │
├──────────────┼───────┼───────────────┼─────────────────────┤
│ § 275.0-2_P2 │   224 │            55 │ 0.24553571428571427 │
│ § 275.0-5_P2 │   167 │            67 │ 0.40119760479041916 │
│ § 275.0-7_P2 │   245 │            51 │ 0.20816326530612245 │
└──────────────┴───────┴───────────────┴─────────────────────┘

In [72]:
query = """
SELECT
    id, count(terms) as term, terms.isLocalScope as isLocalScope
FROM EXTRACTED_TERMS
GROUP BY id, terms.isLocalScope
ORDER BY id
;
"""

conn.sql(query)

┌──────────────┬───────┬──────────────┐
│      id      │ term  │ isLocalScope │
│   varchar    │ int64 │   boolean    │
├──────────────┼───────┼──────────────┤
│ § 275.0-2_P2 │    55 │ true         │
│ § 275.0-2_P2 │   169 │ false        │
│ § 275.0-5_P2 │    66 │ true         │
│ § 275.0-5_P2 │   101 │ false        │
│ § 275.0-7_P2 │    50 │ true         │
│ § 275.0-7_P2 │   195 │ false        │
└──────────────┴───────┴──────────────┘

isLocalScope per id

In [73]:
query = """
WITH TotalCounts AS (
    SELECT
        id,
        COUNT(terms) AS total_terms
    FROM EXTRACTED_TERMS
    GROUP BY id
)
SELECT
    e.id,
    COUNT(e.terms) AS term,
    e.terms.isLocalScope AS isLocalScope,
    (COUNT(e.terms) * 100.0 / tc.total_terms) AS percentage
FROM EXTRACTED_TERMS e
JOIN TotalCounts tc ON e.id = tc.id
GROUP BY e.id, e.terms.isLocalScope, tc.total_terms
ORDER BY e.id, e.terms.isLocalScope;
"""

conn.sql(query)

┌──────────────┬───────┬──────────────┬────────────────────┐
│      id      │ term  │ isLocalScope │     percentage     │
│   varchar    │ int64 │   boolean    │       double       │
├──────────────┼───────┼──────────────┼────────────────────┤
│ § 275.0-2_P2 │   169 │ false        │  75.44642857142857 │
│ § 275.0-2_P2 │    55 │ true         │ 24.553571428571427 │
│ § 275.0-5_P2 │   101 │ false        │  60.47904191616767 │
│ § 275.0-5_P2 │    66 │ true         │  39.52095808383233 │
│ § 275.0-7_P2 │   195 │ false        │  79.59183673469387 │
│ § 275.0-7_P2 │    50 │ true         │ 20.408163265306122 │
└──────────────┴───────┴──────────────┴────────────────────┘

In [39]:
query = """
SELECT
    id, terms, terms.isLocalScope as isLocalScope
FROM EXTRACTED_TERMS
WHERE terms.isLocalScope is NULL
;
"""

conn.sql(query)

┌─────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────┐
│   id    │                                                                            terms                                                                             │ isLocalScope │
│ varchar │ struct(term varchar, definition varchar, confidence double, reason varchar, islocalscope boolean, local_scope_confidence double, local_scope_reason varchar) │   boolean    │
├─────────┴──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┴──────────────┤
│                                                                                        0 rows                                                                                         │
└─────────────────────────────────────────────────────────────────────

## Classify P1

#### True table

In [82]:
upsert_table_from_json(
    conn,
    suffix_key_pattern="classify_P1|true_table",
    prefix_key_pattern="",
    table_name="OPERATIVE_RULES_TRUE",
    source="cfr2sbvr_db/true/documents_true_table.json",
    key_value="classify_p1",
    drop=True,
    content_key="content",
    alias="content",
    doc_id_key="id"
)

classify_P1|true_table
_query_drop_table='\n        DROP TABLE OPERATIVE_RULES_TRUE;\n        '
Trying to drop table because drop parameter is True
Table OPERATIVE_RULES_TRUE dropped
_query_insert_data='\n    INSERT INTO OPERATIVE_RULES_TRUE (id, prompt, file_source, content, created_at)\n    SELECT \n        id, \n        \'classify_p1\' as prompt, \n        \'documents_true_table.json\' as file_source, \n        unnest(content) as content,\n        now() as created_at\n    FROM \n        read_json_auto("temp/documents_true_table.json");\n    '
Trying to insert into OPERATIVE_RULES_TRUE
Catalog Error: Table with name OPERATIVE_RULES_TRUE does not exist!
Did you mean "EXTRACTED_TERMS_TRUE"?
Failed to insert, trying create OPERATIVE_RULES_TRUE
_query_create_table='\n        CREATE TABLE OPERATIVE_RULES_TRUE AS\n        SELECT id, \n        \'classify_p1\' as prompt, \n        \'documents_true_table.json\' as file_source, \n        unnest(content) as content,\n        now() as \'created_

True

In [151]:
query = """
SELECT
    content.doc_id as id,
    content.statement_id,
    content.statement,
    content.sources,
    'documents_true_table.json' as file_source,
    content.type
FROM OPERATIVE_RULES_TRUE
;
"""

true_operative_rules = conn.sql(query).fetchall()

conn.sql(query)

┌───────────┬──────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────┬───────────────────────────┬────────────────┐
│    id     │ statement_id │                                                                                                                                                    statement                                                                                                                                                     │  sources  │        file_source        │      type      │
│  varchar  │   varchar    │                                                                                                                                                     varchar                              

### Predict table

In [91]:
# Define the directory and pattern
directory = Path("cfr2sbvr_db/classification")
pattern = "documents-*.json"

# Use glob to find all files matching the pattern
files = directory.glob(pattern)

drop=True # Drop the table first time
for file_path in files:
    print(file_path)  # Output each file path
    upsert_table_from_json(
        conn,
        suffix_key_pattern="classify_P1|llm_response_classification",
        prefix_key_pattern="",
        table_name="OPERATIVE_RULES",
        source=file_path,
        key_value="classify_p1",
        drop=drop,
        content_key="content",
        alias="content",
        doc_id_key="id"
    )
    drop=False # Stop dropping tables

cfr2sbvr_db/classification/documents-2024-12-08-9.json
classify_P1|llm_response_classification
_query_drop_table='\n        DROP TABLE OPERATIVE_RULES;\n        '
Trying to drop table because drop parameter is True
Catalog Error: Table with name OPERATIVE_RULES does not exist!
Did you mean "OPERATIVE_RULES_TRUE"?
_query_insert_data='\n    INSERT INTO OPERATIVE_RULES (id, prompt, file_source, content, created_at)\n    SELECT \n        id, \n        \'classify_p1\' as prompt, \n        \'documents-2024-12-08-9.json\' as file_source, \n        unnest(content) as content,\n        now() as created_at\n    FROM \n        read_json_auto("temp/documents-2024-12-08-9.json");\n    '
Trying to insert into OPERATIVE_RULES
Catalog Error: Table with name OPERATIVE_RULES does not exist!
Did you mean "OPERATIVE_RULES_TRUE"?
Failed to insert, trying create OPERATIVE_RULES
_query_create_table='\n        CREATE TABLE OPERATIVE_RULES AS\n        SELECT id, \n        \'classify_p1\' as prompt, \n        \

In [106]:
query = """
SELECT
    *
FROM OPERATIVE_RULES
;
"""

conn.sql(query)

┌─────────────┬─────────────┬──────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [97]:
query = """
SELECT
    content.doc_id as id,
    content.statement_id,
    content.statement_title,
    content.statement_text,
    content.statement_sources,
    content.classification
FROM OPERATIVE_RULES
;
"""

conn.sql(query)

┌───────────┬──────────────┬───────────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [142]:
query = """
SELECT 
    content.doc_id as doc_id,
    content.statement_id as statement_id,
    content.statement_title as statement_title,
    content.statement_sources as statement_sources,
    content.file_source as file_source,
    --classification.type as classification_type, 
    --classification.explanation as classification_explanation, 
    --classification.confidence as classification_confidence
    MAX(classification.type) as classification_type, 
    MAX(classification.explanation) as classification_explanation, 
    MAX(classification.confidence) as classification_confidence,
    MAX(file_source) as file_source
FROM (
    SELECT
        file_source,
        content.doc_id,
        content.statement_id,
        content.statement_sources,
        content.statement_title,
        unnest(content.classification) as classification
    FROM
        OPERATIVE_RULES
) AS content
GROUP BY doc_id, statement_id, statement_title, statement_sources, file_source
ORDER BY file_source, doc_id, statement_id
"""

pred_operative_rules = conn.sql(query).fetchall()

conn.sql(query)

┌───────────┬──────────────┬───────────────────────────────────────────┬───────────────────┬──────────────────────────────┬─────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────────────────┬──────────────────────────────┐
│  doc_id   │ statement_id │              statement_title              │ statement_sources │         file_source          │ classification_type │                                                                                       classification_explanation                                                                                        │ classification_confidence │         file_source          │
│  varchar  │   varchar    │                  varchar                  │     varchar[]     │           varchar            │       varchar       │                                         

In [153]:
true_operative_rules

[('§ 275.0-2',
  '3',
  "The Secretary of the Commission (Secretary) will promptly forward a copy to each named party by registered or certified mail at that party's last address filed with the Commission.",
  ['(a)(2)'],
  'documents_true_table.json',
  'Party rules'),
 ('§ 275.0-2',
  '4',
  'If the Secretary certifies that the Commission was served with process, pleadings, or other papers pursuant to paragraph (a)(1) of this section and forwarded these documents to a named party pursuant to paragraph (a)(2) of this section, this certification constitutes evidence of service upon that party.',
  ['(a)(3)'],
  'documents_true_table.json',
  'Party rules'),
 ('§ 275.0-5',
  '1',
  'Notice of the initiation of the proceeding will be published in the Federal Register and will indicate the earliest date upon which an order disposing of the matter may be entered.',
  ['(a)'],
  'documents_true_table.json',
  'Activity rules'),
 ('§ 275.0-5',
  '2',
  'Any interested person may, within the 

In [176]:
pred_dict

{('§ 275.0-2', '3', ('(a)(2)',), 'documents-2024-12-08-1.json'): 'Party rules',
 ('§ 275.0-2', '4', ('(a)(3)',), 'documents-2024-12-08-1.json'): 'Party rules',
 ('§ 275.0-5', '1', ('(a)',), 'documents-2024-12-08-1.json'): 'Party rules',
 ('§ 275.0-5', '2', ('(a)',), 'documents-2024-12-08-1.json'): 'Party rules',
 ('§ 275.0-5', '3', ('(b)',), 'documents-2024-12-08-1.json'): 'Party rules',
 ('§ 275.0-5', '4', ('(c)',), 'documents-2024-12-08-1.json'): 'Party rules',
 ('§ 275.0-2',
  '3',
  ('(a)(2)',),
  'documents-2024-12-08-10.json'): 'Party rules',
 ('§ 275.0-2',
  '4',
  ('(a)(3)',),
  'documents-2024-12-08-10.json'): 'Party rules',
 ('§ 275.0-5', '1', ('(a)',), 'documents-2024-12-08-10.json'): 'Party rules',
 ('§ 275.0-5', '2', ('(a)',), 'documents-2024-12-08-10.json'): 'Party rules',
 ('§ 275.0-5', '3', ('(b)',), 'documents-2024-12-08-10.json'): 'Party rules',
 ('§ 275.0-5', '4', ('(c)',), 'documents-2024-12-08-10.json'): 'Party rules',
 ('§ 275.0-2', '3', ('(a)(2)',), 'documents-20

In [177]:
true_operative_rules

[('§ 275.0-2',
  '3',
  "The Secretary of the Commission (Secretary) will promptly forward a copy to each named party by registered or certified mail at that party's last address filed with the Commission.",
  ['(a)(2)'],
  'documents_true_table.json',
  'Party rules'),
 ('§ 275.0-2',
  '4',
  'If the Secretary certifies that the Commission was served with process, pleadings, or other papers pursuant to paragraph (a)(1) of this section and forwarded these documents to a named party pursuant to paragraph (a)(2) of this section, this certification constitutes evidence of service upon that party.',
  ['(a)(3)'],
  'documents_true_table.json',
  'Party rules'),
 ('§ 275.0-5',
  '1',
  'Notice of the initiation of the proceeding will be published in the Federal Register and will indicate the earliest date upon which an order disposing of the matter may be entered.',
  ['(a)'],
  'documents_true_table.json',
  'Activity rules'),
 ('§ 275.0-5',
  '2',
  'Any interested person may, within the 

In [169]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

# Criar dicionários para fácil acesso, incluindo file_source na chave
pred_dict = {(item[0], item[1], tuple(item[3]), item[4]): item[5] for item in pred_operative_rules}
true_dict = {(item[0], item[1], tuple(item[3])): item[5] for item in true_operative_rules}

# Alinhar e comparar os dados
y_true = []
y_pred = []

# Usar todas as previsões
for item in pred_operative_rules:
    key = (item[0], item[1], tuple(item[3]))
    y_pred.append(item[5])
    y_true.append(true_dict.get(key, 'No Match'))  # 'No Match' se não houver correspondência

# Calcular métricas
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='weighted')

print(f'Precisão: {precision:.2f}')
print(f'Sensibilidade: {recall:.2f}')
print(f'Acurácia: {accuracy:.2f}')
print(f'F1 Score: {f1:.2f}')

Precisão: 0.25
Sensibilidade: 0.50
Acurácia: 0.50
F1 Score: 0.33


/Users/u009165/miniconda3/envs/py3-scorecard/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


The best classification of all checkpoints.

In [123]:
query = """
WITH ExpandedClassifications AS (
    SELECT
        content.doc_id AS id,
        content.statement_id,
        content.statement_title,
        content.statement_text,
        content.statement_sources,
        classification_item.value.type AS type,
        classification_item.value.confidence AS confidence,
        classification_item.value.explanation AS explanation,
        ROW_NUMBER() OVER (PARTITION BY content.doc_id, content.statement_id ORDER BY classification_item.value.confidence DESC) AS rn
    FROM
        OPERATIVE_RULES AS content,
        UNNEST(content.classification) AS classification_item(value)
)
SELECT
    id,
    statement_id,
    statement_title,
    statement_text,
    statement_sources,
    type AS classification_type,
    confidence AS classification_confidence,
    explanation AS classification_explanation
FROM
    ExpandedClassifications
WHERE
    rn = 1;
"""

conn.sql(query)

┌───────────┬──────────────┬───────────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────────┬─────────────────────┬───────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│    id     │ statement_id │              statement_title              │                                                                                                                                                  statement_text                                                                                                                    

#### Evaluate

In [149]:
pred_operative_rules

[('§ 275.0-2',
  '3',
  'Forwarding documents by the Secretary',
  ['(a)(2)'],
  'documents-2024-12-08-1.json',
  'Party rules',
  'This statement specifies the role of the Secretary in forwarding documents to named parties, indicating a responsibility and action linked to a specific role, which is characteristic of a party rule.',
  0.8,
  'documents-2024-12-08-1.json'),
 ('§ 275.0-2',
  '4',
  'Certification of service by the Secretary',
  ['(a)(3)'],
  'documents-2024-12-08-1.json',
  'Party rules',
  'There is a reference to documents and certification as evidence, but the primary focus is on the role and activities, not on data constraints.',
  0.7,
  'documents-2024-12-08-1.json'),
 ('§ 275.0-5',
  '1',
  'Notice of proceeding initiation',
  ['(a)'],
  'documents-2024-12-08-1.json',
  'Party rules',
  'There is no specific reference to roles or responsibilities of parties, making this unlikely to be a party rule.',
  0.9,
  'documents-2024-12-08-1.json'),
 ('§ 275.0-5',
  '2',
  

## Close conn

In [24]:
conn.close()

## Queries

```sql
WITH data AS (
    SELECT 
        json_extract(content, '$.elements[*].verb_symbols') AS verb_symbols
    FROM 
        read_json_auto("temp/data_extracted_elements.json")
)

SELECT 
    unnest(verb_symbols) AS verb_symbol
FROM (
    SELECT 
        json_group_array(json_each_text(verb_symbols)) AS verb_symbols
    FROM 
        data
) AS unnested_data;


CREATE TABLE EXTRACTED_ELEMENTS AS
  SELECT id, 'extract_p1' as prompt, '/Users/u009165/Documents/scorecars/lab/temp/data_extracted_elements.json' as file_source,  unnest(content.elements) as elements FROM read_json_auto("temp/data_extracted_elements.json");


SELECT
·     id,
·     'extract_p1' AS prompt,
·     '/Users/u009165/Documents/scorecars/lab/temp/data_extracted_elements.json' AS file_source,
·     elements.title AS title  -- Extracting the title from elements
· FROM
·     read_json_auto("temp/data_extracted_elements.json"),
‣     unnest(content.elements) AS elements;

SELECT 
    data.id, 
    'extract_p1' AS prompt, 
    '/Users/u009165/Documents/scorecars/lab/temp/data_extracted_elements.json' AS file_source,  
    elem->>'title' AS title,        -- Extracting title from the unnested element
    elem->>'statement' AS statement  -- Extracting statement from the unnested element
FROM 
    read_json_auto("temp/data_extracted_elements.json") AS data,
    unnest(data.content.elements) AS elem; 

SELECT 
    data.id, 
    'extract_p1' AS prompt, 
    '/Users/u009165/Documents/scorecars/lab/temp/data_extracted_elements.json' AS file_source,  
    elem 
FROM 
    read_json_auto("temp/data_extracted_elements.json") AS data,
    unnest(data.content.elements) AS elem; 

  
SELECT 
    data.id, 
    'extract_p1' AS prompt, 
    '/Users/u009165/Documents/scorecars/lab/temp/data_extracted_elements.json' AS file_source,  
    unnest(json_extract(data.content, '$.elements[*].title')) AS titles,
    unnest(json_extract(data.content, '$.elements[*].statement')) AS statements,
    unnest(json_extract(data.content, '$.elements[*].terms')) AS terms,
    unnest(json_extract(data.content, '$.elements[*].verb_symbols')) AS verb_symbols,
    unnest(json_extract(data.content, '$.elements[*].verb_symbols_extracted_confidence')) AS verb_symbols_extracted_confidence,
    unnest(json_extract(data.content, '$.elements[*].classification')) AS classification,
    unnest(json_extract(data.content, '$.elements[*].confidence')) AS confidence,
    unnest(json_extract(data.content, '$.elements[*].reason')) AS reason,
    unnest(json_extract(data.content, '$.elements[*].sources')) AS sources
FROM 
    read_json_auto("temp/data_extracted_elements.json") AS data;

SELECT 
    unnest(json_extract(data.content, '$.elements[*].verb_symbols'), max_depth := 2) AS verb_symbols
FROM 
    read_json_auto("temp/data_extracted_elements.json") AS data;
```